# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
M. Ziegler  ->  M. Ziegler  |  ['M. Ziegler']
J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Martos  ->  G. Martos  |  ['G. Martos']


J. Davies  ->  J. Davies  |  ['J. Davies']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Arxiv has 70 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.19901


extracting tarball to tmp_2506.19901...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.19902


extracting tarball to tmp_2506.19902... done.


...

Found 51 bibliographic references in tmp_2506.20345/aa54675-25corr.bbl.
Error retrieving bib data for Bedell_2018: 'author'
Retrieving document from  https://arxiv.org/e-print/2506.20502


/tmp/ipykernel_3403/2822249172.py:52: LatexWarning: 2506.20502 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2506.20532


extracting tarball to tmp_2506.20532...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.19902-b31b1b.svg)](https://arxiv.org/abs/2506.19902) | **Gravitational Torques from a Lopsided Young Stellar Component Sustain High Black Hole Accretion in NGC 4593**  |
|| N. Winkel, et al. -- incl., <mark>K. Jahnke</mark>, <mark>J. Neumann</mark> |
|*Appeared on*| *2025-06-26*|
|*Comments*| *14 pages, 8 figures, accepted for publication in A&A*|
|**Abstract**|            Supermassive black holes (SMBHs) grow primarily through gas accretion, observed as active galactic nuclei (AGNs). While mergers can drive luminous AGN episodes, secular processes may fuel a substantial portion of cosmic BH growth. Whether these mechanisms can sustain high BH accretion rates remains uncertain. This study aims to identify the secular mechanism driving high BH accretion rates, by targeting a galaxy with moderately massive SMBH, high central gas densities, accretion rates of a few percent of the Eddington limit, and gas kinematics resolved close to black hole-dominated scales. A blind search led to the identification of NGC 4593, which is representative of the AGN population driving BH mass density growth since $z=1$. A prominent single-arm ("$m=1$") molecular gas spiral with ${\rm log}\,M_{\rm mol}/{\rm M}_\odot=8.1\pm0.3$ extends from 1.3$\,$kpc down to the SMBH's sphere of influence ($1.7^{+0.5}_{-0.2}\,{\rm pc}$). Star formation in the spiral is inefficient (${\rm SFR} = 4.9 \times 10^{-2} \,{\rm M}_\odot {\rm /yr}$, $\langle t_{\rm dep}\rangle=3.9 \pm 0.6\,\rm{Gyr}$), whereas inflow rate exceeds the SFR by two orders of magnitude and is sufficient to sustain the current BH accretion rate for $\geq$35$\,$Myr, enabling $\sim$10% SMBH growth. A young, lopsided stellar component (${\rm log}\,M_\star/{\rm M}_\odot=7.5-9.3$) exerts torques on the molecular gas, likely driving the gas inflow. This young stellar component may serve as both a cause and a product of sustained gas funnelling towards the SMBH. These findings provide evidence for the sustained secular $m=1$ feeding mode at high BH accretion rates, linking kpc-scale gas dynamics to the black hole's sphere of influence. This mechanism, consistent with simulation predictions, may represent a key contributor to SMBH growth in luminous AGNs since cosmic noon.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.19932-b31b1b.svg)](https://arxiv.org/abs/2506.19932) | **Follow-Up Exploration of the TWA 7 Planet-Disk System with JWST NIRCam**  |
|| K. A. Crotts, et al. -- incl., <mark>G. Chauvin</mark> |
|*Appeared on*| *2025-06-26*|
|*Comments*| *27 pages, 15 figures, 6 tables, accepted for publication in ApJL*|
|**Abstract**|            The young M-star TWA 7 hosts a bright and near face-on debris disk, which has been imaged from the optical to the submillimeter. The disk displays multiple complex substructures such as three disk components, a large dust clump, and spiral arms, suggesting the presence of planets to actively sculpt these features. The evidence for planets in this disk was further strengthened with the recent detection of a point-source compatible with a Saturn-mass planet companion using JWST/MIRI at 11 $\mu$m, at the location a planet was predicted to reside based on the disk morphology. In this paper, we present new observations of the TWA 7 system with JWST/NIRCam in the F200W and F444W filters. The disk is detected at both wavelengths and presents many of the same substructures as previously imaged, although we do not robustly detect the southern spiral arm. Furthermore, we detect two faint potential companions in the F444W filter at the 2-3$\sigma$ level. While one of these companions needs further followup to determine its nature, the other one coincides with the location of the planet candidate imaged with MIRI, providing further evidence that this source is a sub-Jupiter mass planet companion rather than a background galaxy. Such discoveries make TWA 7 only the second system, after $\beta$ Pictoris, in which a planet predicted by the debris disk morphology has been detected.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.20033-b31b1b.svg)](https://arxiv.org/abs/2506.20033) | **Investigating silicate, carbon, and water in the diffuse interstellar medium: the first shots from WISCI**  |
|| S. Zeegers, et al. -- incl., <mark>J. Bouwman</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-06-26*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            The dusty interstellar medium (ISM) of the Milky Way is distributed in a complex, cloudy structure. It is fundamental to the radiation balance within the Milky Way, provides a reaction surface to form complex molecules, and is the feedstock for future generations of stars and planets. The life cycle of interstellar dust is not completely understood, and neither are its structure nor composition. The abundance, composition, and structure of dust in the diffuse ISM can be determined by combining infrared, optical and ultraviolet spectroscopy. JWST enables measurement of the faint absorption of ISM dust grains against bright stars at kiloparsec distances across the infrared spectrum. Here we present an overview of the project `Webb Investigation of Silicates, Carbons, and Ices' (WISCI) along with interpretation of two targets, GSC 08152-02121 and CPD-59 5831. Observations of 12 WISCI target stars were taken by JWST, the Hubble Space Telescope, Himalayan Chandra Telescope, and the Very Large Telescope. We use these to characterize the targets' spectral types and calculate their line-of-sight extinction parameters, $A_{\rm V}$ and $R_{\rm V}$. We find absorption in the JWST spectra of GSC 08152-02121, and CPD-59 5831 associated with carbonaceous dust around 3.4 and 6.2 micron and amorphous silicates at 9.7 micron. In GSC 08152-02121 we also find indications of absorption by trapped water around 3 micron. This first look from WISCI demonstrates the line-of-sight variability within the sample, and the program's potential to identify and correlate features across ultraviolet to mid-infrared wavelengths.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.20345-b31b1b.svg)](https://arxiv.org/abs/2506.20345) | **Signatures of planets and Galactic subpopulations in solar analogs. Precise chemical abundances with neural networks**  |
|| <mark>G. Martos</mark>, J. Meléndez, L. Spina, S. Lucatello |
|*Appeared on*| *2025-06-26*|
|*Comments*| *Accepted by A&A*|
|**Abstract**|            The aim of this work is to obtain precise atmospheric parameters and chemical abundances automatically for solar twins and analogs to find signatures of exoplanets, as well as to assess how peculiar the Sun is compared to these stars and to analyze any possible fine structures in the Galactic thin disk. We developed a neural network (NN) algorithm using Python to obtain these parameters for a sample of 99 solar twins and solar analogs previously studied in the literature from normalized high-quality spectra from HARPS, with a resolving power of R $\sim$ 115000 and a signal-to-noise ratio S/N > 400. We obtained precise atmospheric parameters and abundance ratios [X/Fe] of 20 chemical elements (Li, C, O, Na, Mg, Al, Si, S, Ca, Sc, Ti, V, Cr, Mn, Co, Ni, Cu, Zn, Y, and Ba). The results are in line with the literature, with average differences and standard deviations of $(2 \pm 27)$ K for T$_{\rm eff}$, $(0.00 \pm 0.06)$ dex for log g, $(0.00 \pm 0.02)$ dex for [Fe/H], $(-0.01 \pm 0.05)$ km s$^{-1}$ for microturbulence velocity, $(0.02 \pm 0.08)$ km s$^{-1}$ for the macro turbulence velocity, and $(-0.12 \pm 0.26)$ km s$^{-1}$ for the projected rotational velocity (vsin$i$). Regarding the chemical abundances, most of the elements agree with the literature within 0.01 - 0.02 dex. The abundances were corrected from the effects of the Galactic chemical evolution and analyzed with the condensation temperature (T$_{\rm cond}$) to verify whether the stars presented depletion of refractories compared to volatiles. We found that the Sun is more depleted in refractory elements compared to volatiles than 89% of the studied solar analogs, with a significance of 9.5$\sigma$ when compared to the stars without detected exoplanets. We also found the possible presence of three subpopulations in the solar analogs: one Cu-rich, one Cu-poor, and the last one slightly older and poor in Na.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.19901-b31b1b.svg)](https://arxiv.org/abs/2506.19901) | **Discovery of Diffuse Radio Emission in a Massive z=1.709 Cool Core Cluster: A Candidate Radio Mini-Halo**  |
|| J. Hlavacek-Larrondo, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-06-26*|
|*Comments*| *17 pages, 7 figures, accepted for publication in ApJL*|
|**Abstract**|            Clusters of galaxies host spectacular diffuse radio sources, extending over scales from 100 kpc to several Mpcs. These sources, with extremely faint surface brightness ($\mu$Jy/arcsec$^2$ level), are not tied to individual galaxies but trace synchrotron emission from large-scale magnetic fields and relativistic particles within the intracluster environment. Here, we report the discovery of a candidate radio mini-halo in SpARCS104922.6+564032.5, the most distant cool-core galaxy cluster identified to date at $z=1.709$, using deep LOFAR 120-168 MHz observations. We show that this emission originates from diffuse cluster-associated processes rather than unresolved AGN or star-forming galaxies. The diffuse radio emission coincides spatially with the X-ray emission of the hot intracluster medium and has a radio power of $P_{\rm 150~MHz}=49.8^{+14.7}_{-11.7} \times10^{24}$ W Hz$^{-1}$, exhibiting striking similarities to low-redshift radio mini-halos. This discovery doubles the redshift of previously known mini-halos, challenging models of inverse Compton losses and indicating the presence of strong magnetic fields, enhanced turbulence in high-redshift clusters, or active hadronic processes that require a cosmic ray-to-thermal energy ratio of 0.07 within 200 kpc, assuming a clumped distribution with spatial correlations among the gas, cosmic rays, and magnetic field that partially compensate for cosmological redshift dimming. It further implies that magnetic fields are efficiently amplified to $\sim$$10~\mu$G levels within a Mpc$^3$ volume during the epoch of cluster formation before $z\sim2$. These findings provide critical insights into high-redshift cluster physics and emphasize the transformative potential of next-generation radio surveys, such as those with the SKA and ngVLA, in exploring the early evolution of galaxy clusters.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.20019-b31b1b.svg)](https://arxiv.org/abs/2506.20019) | **Giant Outer Transiting Exoplanet Mass (GOT 'EM) Survey. VI: Confirmation of a Long-Period Giant Planet Discovered with a Single TESS Transit**  |
|| Z. Essack, et al. -- incl., <mark>M. Ziegler</mark> |
|*Appeared on*| *2025-06-26*|
|*Comments*| *31 pages, 13 figures, 12 tables. Accepted for publication in The Astronomical Journal*|
|**Abstract**|            We report the discovery and confirmation of TOI-4465 b, a $1.25^{+0.08}_{-0.07}~R_{J}$, $5.89\pm0.26~M_{J}$ giant planet orbiting a G dwarf star at $d\simeq$ 122 pc. The planet was detected as a single-transit event in data from Sector 40 of the Transiting Exoplanet Survey Satellite (TESS) mission. Radial velocity (RV) observations of TOI-4465 showed a planetary signal with an orbital period of $\sim$102 days, and an orbital eccentricity of $e=0.24\pm0.01$. TESS re-observed TOI-4465 in Sector 53 and Sector 80, but did not detect another transit of TOI-4465 b, as the planet was not expected to transit during these observations based on the RV period. A global ground-based photometry campaign was initiated to observe another transit of TOI-4465 b after the RV period determination. The $\sim$12 hour-long transit event was captured from multiple sites around the world, and included observations from 24 citizen scientists, confirming the orbital period as $\sim$102 days. TOI-4465 b is a relatively dense ($3.73\pm0.53~\rm{g/cm^3}$), temperate (375-478 K) giant planet. Based on giant planet structure models, TOI-4465 b appears to be enriched in heavy elements at a level consistent with late-stage accretion of icy planetesimals. Additionally, we explore TOI-4465 b's potential for atmospheric characterization, and obliquity measurement. Increasing the number of long-period planets by confirming single-transit events is crucial for understanding the frequency and demographics of planet populations in the outer regions of planetary systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.20532-b31b1b.svg)](https://arxiv.org/abs/2506.20532) | **{\tt RapidGBM}: An Efficient Tool for Fermi-GBM Visibility Checking and Data Analysis with a Case Study of EP240617a**  |
|| <mark>Y. Wang</mark>, et al. |
|*Appeared on*| *2025-06-26*|
|*Comments*| *11 pages, 7 figures, 1 table*|
|**Abstract**|            We have developed a lightweight tool {\tt RapidGBM}, featured by a web-based interface and capabilities of rapid calculation of Fermi-GBM visibilities and performing basic data analysis. It has two key features: (1) immediately check the visibility of Fermi-GBM for new transients, and (2) check the light curve and perform spectral analysis after the hourly TTE data is released. The visibility check and the response matrix generation required for spectral analysis can be achieved through the historical pointing file after the orbit calculation, even when the real-time pointing file is not yet available. As a case, we apply the tool to EP240617a, an X-ray transient triggered by Einstein Probe (EP). We demonstrate the workflow of visibility checking, data processing, and spectral analysis for this event. The results suggest that EP240617a can be classified as an X-ray-rich GRB (XRR) and confirm the feasibility of using historical pointing files for rapid analysis. Further, we discuss possible physical interpretations of such events, including implications for jet launching and progenitor scenarios. Therefore, {\tt RapidGBM} is expected to assist Einstein Probe Transient Advocates (EP-TAs), Space-based multi-band astronomical Variable Objects Monitor Burst Advocates (SVOM-BAs), and other members of the community in cross-checking high-energy transients. Based on prompt emission parameter relations (e.g. $E_{\rm p}$-$E_{\gamma,\rm iso}$), it can also help identify peculiar GRBs (e.g. long-short burst, magnetar giant flare, etc.) and and provide useful references (e.g. more accurate $T_0$) for scheduling follow-up observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.20502-b31b1b.svg)](https://arxiv.org/abs/2506.20502) | **Probing Solar Polar Regions**  |
|| Y. Deng, et al. -- incl., <mark>J. Davies</mark>, <mark>M. Zhang</mark> |
|*Appeared on*| *2025-06-26*|
|*Comments*| *Accepted for publication in Chinese Journal of Space Science*|
|**Abstract**|            The magnetic fields and dynamical processes in the solar polar regions play a crucial role in the solar magnetic cycle and in supplying mass and energy to the fast solar wind, ultimately being vital in controlling solar activities and driving space weather. Despite numerous efforts to explore these regions, to date no imaging observations of the Sun's poles have been achieved from vantage points out of the ecliptic plane, leaving their behavior and evolution poorly understood. This observation gap has left three top-level scientific questions unanswered, 1) How does the solar dynamo work and drive the solar magnetic cycle? 2) What drives the fast solar wind? 3) How do space weather processes globally originate from the Sun and propagate throughout the solar system? The Solar Polar-orbit Observatory (SPO) mission, a solar polar exploration spacecraft, is proposed to address these three unanswered scientific questions by imaging the Sun's poles from high heliolatitudes. In order to achieve its scientific goals, SPO will carry six remote-sensing and four in-situ instruments to measure the vector magnetic fields and Doppler velocity fields in the photosphere, to observed the Sun in the extreme ultraviolet, X-ray, and radio wavelengths, to image the corona and the heliosphere up to 45 $R_\odot$, and to perform in-situ detection of magnetic fields, and low- and high-energy particles in the solar wind.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.19902/./Figures/KinMS_modelling.png', 'tmp_2506.19902/./Figures/time_maps.png', 'tmp_2506.19902/./Figures/lenstronomy_final_qso_fit.png']
copying  tmp_2506.19902/./Figures/KinMS_modelling.png to _build/html/
copying  tmp_2506.19902/./Figures/time_maps.png to _build/html/
copying  tmp_2506.19902/./Figures/lenstronomy_final_qso_fit.png to _build/html/
exported in  _build/html/2506.19902.md
    + _build/html/tmp_2506.19902/./Figures/KinMS_modelling.png
    + _build/html/tmp_2506.19902/./Figures/time_maps.png
    + _build/html/tmp_2506.19902/./Figures/lenstronomy_final_qso_fit.png
found figures ['tmp_2506.19932/./figures/twa7_deconv2.png', 'tmp_2506.19932/./figures/radial_sb.png', 'tmp_2506.19932/./figures/disk_structure.png', 'tmp_2506.19932/./figures/updated_dip_loc.png']
copying  tmp_2506.19932/./figures/twa7_deconv2.png to _build/html/
copying  tmp_2506.19932/./figures/radial_sb.png to _build/html/
copying  tmp_2506.19932/./figures/disk_structure.png to _build/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mbh}{M_{\rm BH}\xspace}$
$\newcommand{\ngc}{NGC 4593\xspace}$</div>



<div id="title">

# Gravitational torques from a lopsided young stellar component sustain high black hole accretion in NGC 4593

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.19902-b31b1b.svg)](https://arxiv.org/abs/2506.19902)<mark>Appeared on: 2025-06-26</mark> -  _14 pages, 8 figures, accepted for publication in A&A_

</div>
<div id="authors">

N. Winkel, et al. -- incl., <mark>K. Jahnke</mark>, <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** Supermassive black holes (SMBHs) grow primarily through gas accretion, observed as active galactic nuclei (AGNs).   While mergers can drive luminous AGN episodes, secular processes may fuel a substantial portion of cosmic black hole growth. Whether these mechanisms can sustain high black hole accretion rates remains uncertain. This study aims to identify the secular mechanism driving high SMBH accretion rates, by targeting a galaxy with moderately massive SMBH, high central gas densities, accretion rates of a few percent of the Eddington limit, and gas kinematics resolved close to black hole-dominated scales. A blind search led to the identification of $\ngc$ , which is representative of the AGN population driving BH mass density growth since $z=1$ . Combining HST imaging, VLT/MUSE spectroscopy, and ALMA imaging, we resolve molecular and ionised gas kinematics close to the sphere of influence of the SMBH. A prominent single-arm (" $m=1$ ”) molecular gas spiral with ${\rm log} M_{\rm mol}/{\rm M}_\odot=8.1\pm0.3$ extends from 1.3 kpc down to the SMBH's sphere of influence ( $1.7^{+0.5}_{-0.2} {\rm pc}$ ).   Star formation in the spiral is inefficient ( ${\rm SFR} = 4.9 \times 10^{-2}  {\rm M}_\odot{\rm /yr}$ , $\langle t_{\rm dep}\rangle=3.9 \pm 0.6 \rm{Gyr}$ ), whereas inflow rate exceeds the SFR by two orders of magnitude and is sufficient to sustain the current SMBH accretion rate for $\geq$ 35 Myr, enabling $\sim$ 10 \% SMBH growth.   A young, lopsided stellar component ( ${\rm log} M_\star/{\rm M}_\odot=7.5-9.3$ ) exerts torques on the molecular gas, likely driving the gas inflow. This young stellar component may serve as both a cause and a product of sustained gas funnelling towards the SMBH. These findings provide evidence for the sustained secular $m=1$ feeding mode at high SMBH accretion rates, linking kpc-scale gas dynamics to the black hole’s sphere of influence. This mechanism, consistent with simulation predictions, may represent a key contributor to SMBH growth in luminous AGNs since cosmic noon.

</div>

<div id="div_fig1">

<img src="tmp_2506.19902/./Figures/KinMS_modelling.png" alt="Fig9" width="100%"/>

**Figure 9. -** 
     Results of the kinematic modelling carried out with \texttt{KinMS}.
     (Left) Surface brightness distribution and line-of-sight velocity field of the CO(2--1) emission, for the low- (bottom) and high-resolution (top) data set respectively.
     Contours correspond to the best-fit model.
     (Right) Position-velocity diagram along the kinematic major axis.
     The model includes circular rotation with radial motions, which dominate the bulk molecular gas kinematics from 1.3 kpc down to the black hole SOI.
 (*fig:KinMS_modelling*)

</div>
<div id="div_fig2">

<img src="tmp_2506.19902/./Figures/time_maps.png" alt="Fig11" width="100%"/>

**Figure 11. -** 
     Time-evolution of the m=1 torque field.
     We simulate the evolution the mass distribution as described in Sect. \ref{subsec:time_evolution_torque_field}, and derive the resulting torque field. After 3 Myr, the center of mass has rotated around the kinematic centre by ?? $^\circ$ (*fig:KinMS_modelling*)

</div>
<div id="div_fig3">

<img src="tmp_2506.19902/./Figures/lenstronomy_final_qso_fit.png" alt="Fig10" width="100%"/>

**Figure 10. -** 
    Photometric decomposition results using \texttt{lenstronomy}.
    (1) HST/ACS HRC F330W flux map with saturated pixels replaced. Grey lines show surface brightness contours of $\Sigma_{\rm mol}$.
    (2) F330W flux map after subtracting the PSF model, revealing a diffuse component near the centre. Dashed contours show the initial guess of the single-Sérsic model fit to this structure.
    (3) F330W residual image after subtracting both the PSF and best-fit single-Sérsic model.
    (4) Residual map normalized by uncertainty.
    (5) Same as (3) but subtracting a 90$^\circ$-rotated PSF from the original image.
    (6) Similar to (4) but the single-Sérsic model was not subtracted.
    The detection of the diffuse component near the AGN does not depend on the PSF subtraction (2 vs 5).
    Aside from compact UV emission from star-forming clumps in the single-arm spiral, no extended UV emission is left over when a single-Sérsic model is subtracted.
 (*fig:lenstronomy_final_qso_fit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.19902"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Follow-Up Exploration of the TWA 7 Planet-Disk System with JWST NIRCam

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.19932-b31b1b.svg)](https://arxiv.org/abs/2506.19932)<mark>Appeared on: 2025-06-26</mark> -  _27 pages, 15 figures, 6 tables, accepted for publication in ApJL_

</div>
<div id="authors">

K. A. Crotts, et al. -- incl., <mark>G. Chauvin</mark>

</div>
<div id="abstract">

**Abstract:** The young M-star TWA 7 hosts a bright and near face-on debris disk, which has been imaged from the optical to the submillimeter. The disk displays multiple complex substructures such as three disk components, a large dust clump, and spiral arms, suggesting the presence of planets to actively sculpt these features. The evidence for planets in this disk was further strengthened with the recent detection of a point-source compatible with a Saturn-mass planet companion using JWST/MIRI at 11 $\mu$ m, at the location a planet was predicted to reside based on the disk morphology. In this paper, we present new observations of the TWA 7 system with JWST/NIRCam in the F200W and F444W filters. The disk is detected at both wavelengths and presents many of the same substructures as previously imaged, although we do not robustly detect the southern spiral arm. Furthermore, we detect two faint potential companions in the F444W filter at the 2-3 $\sigma$ level. While one of these companions needs further followup to determine its nature, the other one coincides with the location of the planet candidate imaged with MIRI, providing further evidence that this source is a sub-Jupiter mass planet companion rather than a background galaxy. Such discoveries make TWA 7 only the second system, after $\beta$ Pictoris, in which a planet predicted by the debris disk morphology has been detected.

</div>

<div id="div_fig1">

<img src="tmp_2506.19932/./figures/twa7_deconv2.png" alt="Fig1.1" width="50%"/><img src="tmp_2506.19932/./figures/radial_sb.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** **Top Left:** MCRDI reduction of TWA 7 in the F200W filter. **Top Right:** The same reduction as in the left panel, but deconvolved. Arrows and labels point to the three ring components that make up the disk, as well as the dip in surface brightness observed in the northwest region of Ring 2. The dashed circle represents the location of the second ring component. **Bottom:** Radial surface brightness profile of the TWA 7 disk between our convolved (light blue) and deconvolved (orange) F200W images compared to the 2017 SPHERE data (black). The three ring locations are highlighted. (*fig:deconv_twa7*)

</div>
<div id="div_fig2">

<img src="tmp_2506.19932/./figures/disk_structure.png" alt="Fig2" width="100%"/>

**Figure 2. -**  Residual maps of TWA 7 in the F200W scaled between -0.5 and 0.5 MJy/sr. **Top:** Red contours represent a surface brightness of 0.12 MJy/sr and blue contours represent a surface brightness of -0.12 MJy/sr in the F200W to highlight the positive and negative residuals that remain after the disk model subtraction. The lower left dashed line represents the location of the dust clump as observed by NIRCam. **Bottom:** Contours represent a surface brightness of 7 and 20 $\mu$Jy arcsec$^{-2}$(0.30 and 0.85 MJy/sr) in the model-subtracted SPHERE data to highlight the location of the spiral arms and dust clump  ([Ren, Choquet and Perrin 2021]()) . The lower left dashed lines represent the location of the dust clump as observed by NIRCam (transparent) and SPHERE (opaque), while the curved dashed line just south of the star represents the location of the spiral arm as observed with SPHERE. We label the residuals similar to [Ren, Choquet and Perrin (2021)](), where S1 and S2 are the south and north spiral arms observed with SPHERE and Clump refers to the southeast dust clump. (*fig:twa7*)

</div>
<div id="div_fig3">

<img src="tmp_2506.19932/./figures/updated_dip_loc.png" alt="Fig3" width="100%"/>

**Figure 3. -** **Top:** Azimuthal surface brightness of Ring 2 as a function of angle in degrees normalized between 0 and 1, although we vertically separate each profile for easier viewing. The surface brightness is measured for our F200W data, as well as the 1998 NICMOS, 2017 SPHERE and 2019 STIS data from [Choquet, Perrin and Chen (2016)]() and  ([Ren, Choquet and Perrin 2021]()) . Additionally we include the best fit cosine curves and colored dashed lines representing the measured dip location. The grey dashed lines represent the dip location measured in [Ren, Choquet and Perrin (2021)]() for reference. **Bottom:** Year each dataset were taken, versus the angular separation of the dip in degrees. The black dashed line and grey shaded region represents the fit with uncertainties to the data measured in [Ren, Choquet and Perrin (2021)](). The three colored lines represent the following scenarios; 1) no change in the dip location (solid purple), 2) a rotation of $0\fdg74$ yr$^{-1}$ based on the best fit linear model to all four data point (dashed pink), and 3) a rotation of $1\fdg20$ yr$^{-1}$ based on the expected rotation of Saturn mass planet at $\sim1$\farcs$5$(solid pink-gold). All errorbars represent 1$\sigma$ uncertainties. (*fig:sb_ring2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.19932"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$</div>



<div id="title">

# Investigating silicate, carbon, and water in the diffuse interstellar medium:\ the first shots from WISCI

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.20033-b31b1b.svg)](https://arxiv.org/abs/2506.20033)<mark>Appeared on: 2025-06-26</mark> -  _Accepted for publication in ApJ_

</div>
<div id="authors">

S. Zeegers, et al. -- incl., <mark>J. Bouwman</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The dusty interstellar medium (ISM) of the Milky Way is distributed in a complex, cloudy structure.It is fundamental to the radiation balance within the Milky Way, provides areaction surface to form complex molecules, and is the feedstock for future generations of stars and planets.The life cycle of interstellar dust is not completely understood, and neither are its structure nor composition.The abundance, composition, and structure of dust in the diffuse ISM can be determined by combining infrared, optical and ultraviolet spectroscopy. _JWST_ enables measurement of the faint absorption of ISM dust grains against bright stars at kiloparsec distances across the infrared spectrum.Here we present an overview of the project `Webb Investigation of Silicates, Carbons, and Ices' (WISCI) along with interpretation of two targets, GSC 08152-02121 and CPD-59 5831.Observations of 12 WISCI target stars were taken by _JWST_ , _the Hubble Space Telescope_ , Himalayan Chandra Telescope, and the Very Large Telescope. We use these to characterize the targets' spectral types and calculate their line-of-sight extinction parameters, $A_{\rm V}$ and $R_{\rm V}$ .We find absorption in the _JWST_ spectra of GSC 08152-02121, and CPD-59 5831 associated with carbonaceous dust around 3.4 and 6.2 $\micron$ and amorphous silicates at 9.7 $\micron$ . In GSC 08152-02121 we also find indications of absorption by trapped water around 3 $\micron$ \. This first look from WISCI demonstrates the line-of-sight variability within the sample, and the program's potential to identify and correlate features across ultraviolet to mid-infrared wavelengths.

</div>

<div id="div_fig1">

<img src="tmp_2506.20033/./2MASSJ085747plot_res_carbon34_2.png" alt="Fig5.1" width="50%"/><img src="tmp_2506.20033/./2MASSJ150958plot_res_carbon34_2.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** NIRCam spectra of the 3.3 \micron aromatic and 3.4 \micron aliphatic hydrocarbon features for GSC 08152-02121 (left) and CPD-59 5831 (right). In the top panel we show the observed spectrum in black, blue dots denote data points from which the continuum was calculated, and the fitted continuum as a green line. The middle panel shows the Gaussian components of the dust model as orange (aliphatic) and purple (aromatic) dashed lines, with the total denoted by a red solid line, all in units of optical depth. The stellar line (Hydrogen Pfund-$\delta$) contaminating the model fit at 3.3 $\micron$ is denoted by a cyan solid line. The gray contours behind the data indicate the $1\sigma$ statistical uncertainty derived from the continuum. The bottom panel shows the significance of the residuals ($\pm2.5\sigma$) after subtraction of the continuum and line model, and normalization by the standard deviation.
    Shortward of the 3.3 $\micron$ aromatic and 3.4 $\micron$ aliphatic hydrocarbon features the GSC 08152-02121 spectrum shows an absorption feature which may be attributed to trapped water. No comparable absorption is seen at the same wavelength range in the CPD-59 5831 spectrum. (*fig:c34_absorption*)

</div>
<div id="div_fig2">

<img src="tmp_2506.20033/./Topdown_IntegExt_wsun_10lac.png" alt="Fig2.1" width="50%"/><img src="tmp_2506.20033/./Gaia_galaxy3.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** _Top_: Extinction map of the local ISM viewed top-down taken from \citet{Dharmawardena24}. The Sun (yellow circle) is at $X$, $Y$ = 0, 0 and the Galactic centre is at $X$, $Y$ = 8.2, 0 kpc. The twelve targets (cf. Table \ref{tab:sources}) are represented by colored stars. The two stars analyzed in this paper are shown in red and the other ten are shown in blue. The small gray star near the center of this figure indicates 10 Lac ($A_{\rm V} = 0.21 $mag), which was used as a reference star. The contours represent extinction in steps of 0.2 mag.
_Bottom_: Dust map of the Milky Way galactic plane derived from Gaia photometry (see \citet{Andrae:23} for more details) with the twelve targets following the same color scheme as above. Note that some of the targets overlap in this projection.
 (*fig:map*)

</div>
<div id="div_fig3">

<img src="tmp_2506.20033/./2MASSJ085747plot_res_carbon62_0.png" alt="Fig6.1" width="50%"/><img src="tmp_2506.20033/./2MASSJ150958plot_res_carbon62_0.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Here we show the MIRI MRS spectrum around 6.2 \micron for GSC 08152-02121 (left) and CPD-59 5831 (right). In the top panel we show the observed spectrum in black with the fitted continuum as a green line. Blue data points denote the parts of the spectrum used to determine the continuum. An unidentified feature is present around 5.8 \micron which is excluded from the continuum. The middle panel shows the Gaussian dust components of the model fit in units of optical depth. The two components are shown by dashed lines and the total dust component is shown by the red line. The cyan line indicates the position and strength of a stellar line (Hydrogen $n=7$ to $n=13$). The gray contours behind the data indicate the $1\sigma$ statistical uncertainty derived from the continuum. The bottom panel shows the significance of the residuals ($\pm5\sigma$) after subtraction of the continuum and line model, and normalization by the standard deviation. (*fig:c62_absorption*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.20033"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{.8}$
$\newcommand{\arraystretch}{.8}$
$\newcommand{\arraystretch}{0.1}$
$\newcommand{\arraystretch}{0.1}$</div>



<div id="title">

# Signatures of planets and Galactic subpopulations in solar analogs: Precise chemical abundances with neural networks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.20345-b31b1b.svg)](https://arxiv.org/abs/2506.20345)<mark>Appeared on: 2025-06-26</mark> -  _Accepted by A&A_

</div>
<div id="authors">

<mark>G. Martos</mark>, J. Meléndez, L. Spina, a. S. Lucatello

</div>
<div id="abstract">

**Abstract:**            The aim of this work is to obtain precise atmospheric parameters and chemical abundances automatically for solar twins and analogs to find signatures of exoplanets, as well as to assess how peculiar the Sun is compared to these stars and to analyze any possible fine structures in the Galactic thin disk. We developed a neural network (NN) algorithm using Python to obtain these parameters for a sample of 99 solar twins and solar analogs previously studied in the literature from normalized high-quality spectra from HARPS, with a resolving power of R $\sim$ 115000 and a signal-to-noise ratio S/N > 400. We obtained precise atmospheric parameters and abundance ratios [X/Fe] of 20 chemical elements (Li, C, O, Na, Mg, Al, Si, S, Ca, Sc, Ti, V, Cr, Mn, Co, Ni, Cu, Zn, Y, and Ba). The results are in line with the literature, with average differences and standard deviations of $(2 \pm 27)$ K for T$_{\rm eff}$, $(0.00 \pm 0.06)$ dex for log g, $(0.00 \pm 0.02)$ dex for [Fe/H], $(-0.01 \pm 0.05)$ km s$^{-1}$ for microturbulence velocity, $(0.02 \pm 0.08)$ km s$^{-1}$ for the macro turbulence velocity, and $(-0.12 \pm 0.26)$ km s$^{-1}$ for the projected rotational velocity (vsin$i$). Regarding the chemical abundances, most of the elements agree with the literature within 0.01 - 0.02 dex. The abundances were corrected from the effects of the Galactic chemical evolution and analyzed with the condensation temperature (T$_{\rm cond}$) to verify whether the stars presented depletion of refractories compared to volatiles. We found that the Sun is more depleted in refractory elements compared to volatiles than 89% of the studied solar analogs, with a significance of 9.5$\sigma$ when compared to the stars without detected exoplanets. We also found the possible presence of three subpopulations in the solar analogs: one Cu-rich, one Cu-poor, and the last one slightly older and poor in Na.         

</div>

<div id="div_fig1">

<img src="tmp_2506.20345/./fig/err_elem_sigma_bedell.png" alt="Fig1" width="100%"/>

**Figure 1. -** Median error of the abundance ratios [X/Fe] of the elements versus the standard deviation of the residuals when comparing our automatic abundances with Bedell_2018 (*fig:err_elem_sigma_bedell*)

</div>
<div id="div_fig2">

<img src="tmp_2506.20345/./fig/thickdisk_feh.png" alt="Fig2" width="100%"/>

**Figure 2. -** Separation of the stars of the sample between thin and thick disk according to their [Mg/Fe] ratio in relation to [Fe/H]. The stars above the red line and represented by blue crosses belong to the thick disk. (*fig:sep_disks*)

</div>
<div id="div_fig3">

<img src="tmp_2506.20345/./fig/resid_Z.png" alt="Fig3" width="100%"/>

**Figure 3. -** Standard deviation of the residuals of the fit of [X/Fe] versus age and median error according to the atomic number (Z) of the elements. (*fig:std_Z*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.20345"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

544  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
